# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [8]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('/Users/maraqawalid/scaling_to_production/05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [18]:
# Create X dataframe (features)
X = adult_dt.drop('income', axis=1)

# Create Y dataframe (target)
Y = adult_dt['income']

# Display X and Y shapes to verify
print('Shape of X:', X.shape)
print('Shape of Y:', Y.shape)


Shape of X: (32561, 14)
Shape of Y: (32561,)
100    1
101    1
102    0
103    0
104    0
105    1
106    0
107    0
108    0
109    0
110    0
111    1
112    1
113    0
114    0
115    0
116    0
117    1
118    0
119    0
120    0
Name: income, dtype: int64


In [19]:
from sklearn.model_selection import train_test_split

# Split data into training and testing sets (70-30 split, random_state=42)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Display the shapes of the resulting splits
print('Shape of X_train:', X_train.shape)
print('Shape of X_test:', X_test.shape)
print('Shape of Y_train:', Y_train.shape)
print('Shape of Y_test:', Y_test.shape)

Shape of X_train: (22792, 14)
Shape of X_test: (9769, 14)
Shape of Y_train: (22792,)
Shape of Y_test: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

Random state is key to producing reproducible results, as it ensures that the dataset is split in the same way into training and test subsets.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [45]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Assuming X_train and X_test are your feature DataFrames

# Numerical features: columns that are not categorical
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns

# Categorical features: columns that are 'object' type
categorical_features = X_train.select_dtypes(include=['object']).columns

# Create transformers
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', StandardScaler())
])

# Custom function to apply pd.get_dummies to categorical columns
def pd_get_dummies(df, columns):
    return pd.get_dummies(df, columns=columns, drop_first=True)

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', 'passthrough', categorical_features)  # Use 'passthrough' to keep categorical columns unchanged
    ])

# Fit and transform the data
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Convert transformed data to DataFrame
# For X_train_transformed
X_train_transformed_df = pd.DataFrame(X_train_transformed, columns=list(numerical_features) + list(categorical_features))
# For X_test_transformed
X_test_transformed_df = pd.DataFrame(X_test_transformed, columns=list(numerical_features) + list(categorical_features))

# Apply pd.get_dummies to categorical columns in the transformed DataFrames
X_train_transformed_df = pd_get_dummies(X_train_transformed_df, columns=categorical_features)
X_test_transformed_df = pd_get_dummies(X_test_transformed_df, columns=categorical_features)

# Display the shapes for verification
print("X_train_transformed_df shape:", X_train_transformed_df.shape)
print("X_test_transformed_df shape:", X_test_transformed_df.shape)

# Display the transformed data
print(X_train_transformed_df.head())
print(X_test_transformed_df.head())


X_train_transformed_df shape: (22792, 100)
X_test_transformed_df shape: (9769, 98)
        age    fnlwgt education-num capital-gain capital-loss hours-per-week  \
0 -0.334771 -1.261292     -0.423219    -0.146349     5.136118      -2.309851   
1  0.691223  0.510642     -0.035377     0.929742    -0.219921      -0.031295   
2 -1.140909 -1.261841     -0.035377    -0.146349    -0.219921      -0.845065   
3  1.277505  0.622115     -0.423219    -0.146349    -0.219921      -0.031295   
4 -1.580621  1.018212     -1.198902    -0.146349    -0.219921      -2.553982   

   workclass_ Federal-gov  workclass_ Local-gov  workclass_ Never-worked  \
0                   False                 False                    False   
1                   False                 False                    False   
2                   False                 False                    False   
3                   False                  True                    False   
4                   False                 False         

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [55]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd

# Assuming X_train and X_test are your feature DataFrames
# Assuming Y_train and Y_test are your target Series or arrays

# Define numerical and categorical features
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

# Pipeline for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', StandardScaler())
])

# Pipeline for categorical features using OneHotEncoder
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with most frequent
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # OneHotEncoder for categorical variables
])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Pipeline combining preprocessing and RandomForestClassifier
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Fit the pipeline on the training data
pipeline.fit(X_train, Y_train)

# Predictions
Y_pred = pipeline.predict(X_test)

# Evaluate the model
from sklearn.metrics import classification_report, accuracy_score

print("Accuracy:", accuracy_score(Y_test, Y_pred))
print("\nClassification Report:")
print(classification_report(Y_test, Y_pred))


Accuracy: 0.8548469648889344

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.92      0.91      7455
           1       0.72      0.63      0.67      2314

    accuracy                           0.85      9769
   macro avg       0.81      0.78      0.79      9769
weighted avg       0.85      0.85      0.85      9769



# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [56]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, balanced_accuracy_score, roc_auc_score, log_loss
import pandas as pd

# Assuming X_train and X_test are your feature DataFrames
# Assuming Y_train and Y_test are your target Series or arrays

# Define numerical and categorical features
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

# Pipeline for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', StandardScaler())
])

# Pipeline for categorical features using OneHotEncoder
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with most frequent
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # OneHotEncoder for categorical variables
])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Pipeline combining preprocessing and RandomForestClassifier
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Define metrics to measure during cross-validation
scoring = {
    'log_loss': make_scorer(log_loss, greater_is_better=False, needs_proba=True),
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}

# Perform cross-validation
cv_results = cross_validate(pipeline, X_train, Y_train, scoring=scoring, cv=5, n_jobs=-1)

# Extract and print results
print("Negative Log Loss (mean):", -cv_results['test_log_loss'].mean())
print("ROC AUC (mean):", cv_results['test_roc_auc'].mean())
print("Accuracy (mean):", cv_results['test_accuracy'].mean())
print("Balanced Accuracy (mean):", cv_results['test_balanced_accuracy'].mean())


/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/metrics/_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


Negative Log Loss (mean): 0.3535968297855201
ROC AUC (mean): 0.9037924780402848
Accuracy (mean): 0.8520098391129668
Balanced Accuracy (mean): 0.7735260515158988


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [57]:
results_df = pd.DataFrame({
    'Fold': range(1, 6),
    'Negative Log Loss': -cv_results['test_log_loss'],
    'ROC AUC': cv_results['test_roc_auc'],
    'Accuracy': cv_results['test_accuracy'],
    'Balanced Accuracy': cv_results['test_balanced_accuracy']
})

# Sort results by Negative Log Loss
results_df_sorted = results_df.sort_values(by='Negative Log Loss')

# Display sorted results
print("Fold-level results sorted by Negative Log Loss:")
print(results_df_sorted)

Fold-level results sorted by Negative Log Loss:
   Fold  Negative Log Loss   ROC AUC  Accuracy  Balanced Accuracy
4     5           0.345979  0.902568  0.854761           0.775220
0     1           0.348716  0.905604  0.851064           0.776618
1     2           0.355970  0.901597  0.844703           0.765044
3     4           0.357709  0.906501  0.856955           0.777899
2     3           0.359610  0.902692  0.852567           0.772849


Calculate the mean of each metric. 

In [ ]:
mean_metrics = results_df.mean()

# Display mean metrics
print("Mean metrics across all folds:")
print(mean_metrics)

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [61]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score
import pandas as pd

# Assuming X_train and X_test are your feature DataFrames
# Assuming Y_train and Y_test are your target Series or arrays

# Define numerical and categorical features
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

# Pipeline for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', StandardScaler())
])

# Pipeline for categorical features using OneHotEncoder
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with most frequent
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # OneHotEncoder for categorical variables
])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Pipeline combining preprocessing and RandomForestClassifier
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Fit the pipeline on the training data
pipeline.fit(X_train, Y_train)

# Predict probabilities on the test set
Y_pred_proba = pipeline.predict_proba(X_test)

# Calculate negative log loss
logloss = log_loss(Y_test, Y_pred_proba)

# Calculate ROC AUC
roc_auc = roc_auc_score(Y_test, Y_pred_proba[:, 1])

# Calculate accuracy
accuracy = accuracy_score(Y_test, pipeline.predict(X_test))

# Calculate balanced accuracy
balanced_accuracy = balanced_accuracy_score(Y_test, pipeline.predict(X_test))

# Print the results
print(f"Negative Log Loss: {logloss:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Balanced Accuracy: {balanced_accuracy:.4f}")


Negative Log Loss: 0.3683
ROC AUC: 0.9002
Accuracy: 0.8551
Balanced Accuracy: 0.7763


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

We recoded because it's easier to treat income as a class, since incomes could have many unique values for such a large dataset. It's simpler therefore, to treat adults who make more than 50K a year as "high-income" class and the others as a base class

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.